In [1]:
import requests
import re
import socket

def verificar_scielo_por_titulo_ano(titulo, ano):
    """
    Verifica se um artigo está indexado na SciELO usando web scraping (busca por título e ano).

    Args:
        titulo (str): O título do artigo.
        ano (int): O ano de publicação do artigo.

    Returns:
        bool: True se o artigo for encontrado na SciELO, False caso contrário.
    """

    # URL de busca da SciELO (inicialmente com o nome de domínio)
    search_url = "https://search.scielo.org/?" 
    query_params = {
        "q": titulo,
        # "lang": "pt",  # Para filtrar por linguagem, conforme necessário
        "count": 10,      # Número de resultados a serem buscados (ajuste conforme necessário)
        "from": 0,
        "output": "site"
    }

    # Definir headers e user-agent
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3' 
 
    }

    try:
        # Incluir os headers na requisição
        response = requests.get(search_url, params=query_params, headers=headers)

    except requests.exceptions.RequestException as e:
        if isinstance(e, requests.exceptions.ConnectionError) and "Failed to resolve" in str(e):
            # Se o erro for de resolução de nome, tentar usar o endereço IP
            try:
                ip_address = socket.gethostbyname('search.scielo.org')
                search_url = f"https://{ip_address}/"  # Substituir o nome de domínio pelo IP
                response = requests.get(search_url, params=query_params, headers=headers)
            except socket.gaierror as e:
                print(f"Erro ao obter o endereço IP da SciELO: {e}")
                return False
        else:
            # Outro tipo de erro na requisição
            print(f"Erro na requisição à SciELO: {e}")
            return False

    # Encontrar a seção de resultados
    resultados_section_match = re.search(r'<section>.*?<div id="ResultArea">.*?<div class="results">(.*?)</div>', response.text, re.DOTALL)
    print('Seção:')
    print(resultados_section_match)

    if not resultados_section_match:
        print("Seção de resultados não encontrada.")
        return False

    resultados_section = resultados_section_match.group(1)
    print(resultados_section)

    # Expressão regular corrigida para capturar todos os elementos filhos dentro de cada resultado
    resultados = re.findall(r'<div class="item"[^>]*?>(.*?)</div>', resultados_section, re.DOTALL)

    print(resultados)  # Imprimir os resultados para verificar se a captura está funcionando

    for resultado in resultados:
        # Expressão regular para capturar o título dentro do resultado
        titulo_match = re.search(r'<strong class="title"[^>]*?>(.*?)</strong>', resultado, re.DOTALL)
        if titulo_match and titulo_match.group(1).lower().strip() == titulo.lower():
            # Expressão regular para capturar o ano dentro do resultado
            ano_match = re.search(r'(\d{4})', resultado)  # Busca qualquer sequência de 4 dígitos (ano)
            if ano_match and ano_match.group(1).strip() == str(ano):
                return True

    return False

In [ ]:
doi = "https://doi.org/10.1590/S0104-11692008000100023"
titulo_do_artigo = "Aluminum as a risk factor for Alzheimer's disease"
ano_de_publicacao = "2008"

verificar_scielo_por_titulo_ano(titulo_do_artigo, ano_de_publicacao)